# Chr17-22, GTEx Adipose (subcutaneous) fusion weights

## Part1: simulation and mr.ash

* set up

In [1]:
wkdir = "/project2/mstephens/causalTWAS/simulations/simulation_ashtest_20200721/"

In [2]:
codedir = "/project2/mstephens/causalTWAS/causal-TWAS/code/"

In [3]:
geno = "/project2/mstephens/causalTWAS/ukbiobank/ukb_chr17to22_s20000"

In [4]:
weight = "/project2/mstephens/causalTWAS/fusion_weights/Adipose_Subcutaneous"

In [5]:
maintag = '20200721-3'

In [24]:
subtags =  [str(i) for i in list(range(10,101))]

* prepare param 

In [25]:
cd {wkdir}

/project2/mstephens/causalTWAS/simulations/simulation_ashtest_20200721


In [26]:
template = "{wkdir}/param-{maintag}.R".format(wkdir = wkdir, maintag = maintag)

In [27]:
a = open(template, 'r').readlines()

In [28]:
for subtag in subtags:
    with open("param-{maintag}-{subtag}.R".format(maintag = maintag, subtag = subtag), 'w') as newpar:
        a[5] = 'SED <- {sed}\n'.format(sed = str(950 + int(subtag)))
        newpar.writelines(a)

* commands

In [29]:
g2dir = "cd {wkdir}"

In [30]:
simu = "Rscript {codedir}/run_simulate_data.R {geno} {weight} param-{tag}.R simu_{tag}"

In [31]:
simu_expr = "{wkdir}/simu_{tag}-cis-expr.Rd"

In [32]:
simu_pheno = "{wkdir}/simu_{tag}-pheno.Rd"

In [33]:
mrash2 = "Rscript {codedir}/run_test_mr.ash2s.R {geno} {simu_expr} {simu_pheno} {tag}"

In [34]:
exprgwas = "Rscript {codedir}/run_gwas_expr.R {simu_expr} {simu_pheno} {tag}"

In [35]:
snpgwas = "Rscript {codedir}/run_gwas_snp.R {geno} {simu_pheno} {tag}"

In [36]:
ld = "Rscript {codedir}/run_ld.R {geno} {simu_expr} {simu_pheno} {tag}"

* write master file

In [37]:
tags = [maintag + '-' + subtag for subtag in subtags]

In [38]:
masterfile = 'master_run3-x.sh'

In [39]:
cd {wkdir}

/project2/mstephens/causalTWAS/simulations/simulation_ashtest_20200721


In [40]:
with open(masterfile, 'w') as outf:
    for tag in tags:
        outf.write(';'.join([
              'source activate twas', 
              g2dir,
              simu,
              #mrash2,
              #exprgwas,
              #snpgwas,
              #ld
        ]).format(
              simu_expr = simu_expr,
              simu_pheno = simu_pheno,
              wkdir = wkdir,
              codedir = codedir,
              geno = geno,
              weight = weight,
              tag = tag
        ).format(
              wkdir = wkdir,
              tag = tag
        ) + '\n')

In [41]:
!python /home/simingz/cancer_somatic/cancer_somatic/annotation/pMake_batchjobs_RCC.py {masterfile} --mem 16 --ntasks 6 --partition broadwl

planning to run 91 jobs ...
Type: sbatch submit_parallel_master_run3-x.sh.sh


## Part 2: susie

* set up 

In [87]:
susiedir = "/project2/mstephens/causalTWAS/simulations/simulation_susietest_20200721"

In [88]:
mrashdir = wkdir
mrashres = "{mrashdir}/{tag}-mr.ash2s.{tag2}"
prior = "{mrashdir}/{tag}-mr.ash2s.{tag2}.param.txt"

* commands

In [89]:
g2susiedir = "cd {susiedir}"
susie = "Rscript {codedir}/run_test_susie.R {geno} {simu_expr} {simu_pheno} {prior} {mrashres} {tag}.{tag2} 3"

* write commands

In [90]:
cd {susiedir}

/project2/mstephens/causalTWAS/simulations/simulation_susietest_20200721


In [91]:
susietags = [maintag + '-' + str(subtag) for subtag in range(1,51)]
tag2s = ['zerose-es', 'lassoes-se', 'zeroes-es', 'lassoes-es']

In [92]:
susiemasterfile = 'susie_run1-x.sh'

In [93]:
with open(susiemasterfile, 'w') as outf:
    for tag in susietags:
        for tag2 in tag2s:
            outf.write(';'.join([
                'source activate twas', 
                g2susiedir,
                susie
            ]).format(
                susiedir = susiedir,
                codedir = codedir,
                geno = geno,
                simu_expr = simu_expr,
                simu_pheno = simu_pheno,
                prior = prior,
                mrashres = mrashres,
                tag = tag, 
                tag2 = tag2,
            ).format(
                wkdir = wkdir,
                mrashdir = mrashdir,
                tag = tag, 
                tag2 = tag2,
            ) + '\n')

In [94]:
!python /home/simingz/cancer_somatic/cancer_somatic/annotation/pMake_batchjobs_RCC.py {susiemasterfile} --mem 28 --ntasks 1 --partition broadwl

planning to run 200 jobs ...
Type: sbatch submit_parallel_susie_run1-x.sh.sh


## Part3: susie, fix prior

In [103]:
susiedir = "/project2/mstephens/causalTWAS/simulations/simulation_susietest_20200721/20200721-1-fixprior_rpip0.5"

In [104]:
mrashdir = wkdir
mrashres = "{mrashdir}/{tag}-mr.ash2s.lassoes-es"
prior = "{susiedir}/20200721-1-fixedprior_{partag}.txt"
susiemaintag = "20200721-1"

* commands

In [105]:
g2susiedir = "cd {susiedir}"
susie = "Rscript {codedir}/run_test_susie_temp.R {geno} {simu_expr} {simu_pheno} {prior} {mrashres} {tag}.fixedprior{partag} 1"

* write commands

In [106]:
cd {susiedir}

/project2/mstephens/causalTWAS/simulations/simulation_susietest_20200721/20200721-1-fixprior_rpip0.5


In [107]:
susietags = [susiemaintag + '-' + str(subtag) for subtag in range(1,51)]
partags = ['1', '2', '3']

In [108]:
susiemasterfile = 'susie_run1-x-fixedp.sh'

In [109]:
with open(susiemasterfile, 'w') as outf:
    for tag in susietags:
        for partag in partags:
            outf.write(';'.join([
                'source activate twas', 
                g2susiedir,
                susie
            ]).format(
                susiedir = susiedir,
                codedir = codedir,
                geno = geno,
                simu_expr = simu_expr,
                simu_pheno = simu_pheno,
                prior = prior,
                mrashdir = mrashdir,
                mrashres = mrashres,
                tag = tag,
                partag = partag
            ).format(
                mrashdir = mrashdir,
                wkdir = wkdir,
                susiedir = susiedir,
                tag = tag,
                partag = partag
            ) + '\n')

In [112]:
!python /home/simingz/cancer_somatic/cancer_somatic/annotation/pMake_batchjobs_RCC.py {susiemasterfile} --mem 28 --ntasks 1 --partition broadwl

planning to run 150 jobs ...
Type: sbatch submit_parallel_susie_run1-x-fixedp.sh.sh
